In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [17]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from nhdp import nestedHierarchicalNeuralTopicModel
from tsgntm import TreeStructuredGaussianNeuralTopicModel
from tree import get_descendant_idxs
from evaluation import get_hierarchical_affinity, get_topic_specialization, print_topic_sample, print_topic_year
from configure import get_config

pd.set_option('display.max_columns', 30)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# load data & set config

In [3]:
config = get_config(nb_name)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [6]:
def debug(variable, sample_batch=None):
    if sample_batch is None: sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch, mode='test')
    _variable = sess.run(variable, feed_dict=feed_dict)
    return _variable

def check(variable):
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    sample_batch = test_batches[0]
    feed_dict = model.get_feed_dict(sample_batch, mode='test', assertion=True)
    _variable = sess.run(variable, feed_dict=feed_dict)
    return _variable

# model

## initialize model

In [7]:
if 'sess' in globals(): sess.close()
TopicModels = {'hntm': HierarchicalNeuralTopicModel, 'nhdp': nestedHierarchicalNeuralTopicModel, 'tsgntm': TreeStructuredGaussianNeuralTopicModel}
TopicModel = TopicModels[config.model]
model = TopicModel(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

# run

## initialize log

In [8]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','','VALID:','','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL', 'GAUSS', 'REG','LOSS','PPL','NLL','KL', 'GAUSS','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))
    
def validate(sess, batches, model):
    losses = []
    ppl_list = []
    prob_topic_list = []
    n_bow_list = []
    n_topics_list = []
    for ct, batch in batches:
        feed_dict = model.get_feed_dict(batch, mode='test')
        loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_gauss_batch, topic_loss_reg_batch, ppls_batch, prob_topic_batch, n_bow_batch, n_topics_batch \
            = sess.run([model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_gauss, model.topic_loss_reg, model.topic_ppls, model.prob_topic, model.n_bow, model.n_topics], feed_dict = feed_dict)
        losses += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_gauss_batch, topic_loss_reg_batch]]
        ppl_list += list(ppls_batch)
        prob_topic_list.append(prob_topic_batch)
        n_bow_list.append(n_bow_batch)
        n_topics_list.append(n_topics_batch)
    loss_mean, topic_loss_recon_mean, topic_loss_kl_mean, topic_loss_gauss_mean, topic_loss_reg_mean = np.mean(losses, 0)
    ppl_mean = np.exp(np.mean(ppl_list))
    
    probs_topic = np.concatenate(prob_topic_list, 0)
    
    n_bow = np.concatenate(n_bow_list, 0)
    n_topics = np.concatenate(n_topics_list, 0)
    probs_topic_mean = np.sum(n_topics, 0) / np.sum(n_bow)
    
    return loss_mean, topic_loss_recon_mean, topic_loss_kl_mean, topic_loss_gauss_mean, topic_loss_reg_mean, ppl_mean, probs_topic_mean    

## train & validate model

In [13]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_gauss_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_gauss, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_gauss_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

        if global_step_log % config.log_period == 0:
            # validate
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_gauss_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_gauss_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)

            # test
            if ppl_dev < ppl_min:
                ppl_min = ppl_dev
                loss_test, _, _, _, _, ppl_test, _ = validate(sess, test_batches, model)
                saver.save(sess, config.path_model, global_step=global_step_log)
                cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
                update_checkpoint(config, checkpoint, global_step_log)
            
            # visualize topic
            topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
            topics_freq_idxs = bow_idxs[topics_freq_indices]
            topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
            topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
            descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
            recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
            
            depth_specs = get_topic_specialization(sess, model, instances_test)
            hierarchical_affinities = get_hierarchical_affinity(sess, model)
            
            # log
            clear_output()
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_gauss_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_gauss_dev, '%.2f'%topic_loss_reg_dev, \
                    '%.2f'%loss_test, '%.0f'%ppl_test, \
                    '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                    '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)
            cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
            print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)
            print(np.sum(debug(model.topic_logvars), 1))
            
            # update tree
            if not config.static:
                config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
                if update_tree_flg:
                    print(config.tree_idxs)
                    name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                    if 'sess' in globals(): sess.close()
                    model = HierarchicalNeuralTopicModel(config)
                    sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                    name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                    sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                    saver = tf.train.Saver(max_to_keep=1)
                
            time_start = time.time()

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)
topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
topics_freq_idxs = bow_idxs[topics_freq_indices]
topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
display(log_df)
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

TRAIN:                                      VALID:  \
      Time   Ep   Ct     LOSS   PPL      NLL     KL  GAUSS   REG     LOSS   
5000    78    8  463  9296.33  2037  9261.69  14.30  20.34  0.14  9258.74   
10000   74   17  360  9242.14  1938  9199.85  16.53  25.75  0.09  9215.13   
15000   48   26  257  9213.92  1890  9168.31  17.47  28.17  0.07  9202.35   
20000   73   35  154  9196.61  1862  9149.24  17.97  29.42  0.06  9196.60   
25000   58   44   52  9184.68  1843  9136.24  18.28  30.18  0.06  9194.21   
30000   74   52  516  9176.03  1829  9126.91  18.49  30.68  0.05  9191.57   
35000   73   61  413  9169.44  1818  9119.86  18.65  31.02  0.05  9187.20   
40000   73   70  310  9164.03  1810  9114.23  18.76  31.25  0.05  9186.36   
45000   74   79  207  9159.41  1803  9109.41  18.86  31.41  0.05  9185.94   
50000   73   88  104  9155.50  1797  9105.32  18.94  31.52  0.05  9184.84   
55000   74   97    1  9152.06  1792  9101.70  19.00  31.60  0.05  9181.40   
60000   73  105  465  9149.18  1788  9098.68  19.05  31.65  0.05  9182.45   
65000   75  114  362  9146.74  1784  9096.11  19.10  31.69  0.05  9179.98   
70000   76  123  259  9144.47  1780  9093.77  19.14  31.71  0.05  9181.39   
75000   75  132  156  9142.41  1777  9091.63  19.18  31.73  0.05  9179.02   

                                            TEST:       SPEC:              \
        PPL      NLL     KL  GAUSS   REG     LOSS   PPL     1     2     3   
5000   1906  9215.00  17.15  26.59  0.05  9258.81  1905  0.14  0.20  0.34   
10000  1829  9162.98  18.97  33.18  0.04  9214.32  1829  0.16  0.27  0.41   
15000  1811  9150.28  19.29  32.77  0.04  9203.23  1813  0.11  0.29  0.44   
20000  1801  9144.04  19.38  33.19  0.03  9198.12  1804  0.15  0.26  0.44   
25000  1799  9141.66  19.25  33.30  0.04  9191.46  1794  0.15  0.31  0.44   
30000  1796  9139.42  19.34  32.81  0.04  9187.63  1789  0.18  0.29  0.46   
35000  1788  9134.71  19.44  33.04  0.04  9187.27  1788  0.13  0.31  0.44   
40000  1788  9133.93  19.49  32.93  0.04  9186.22  1787  0.10  0.32  0.45   
45000  1787  9133.62  19.52  32.80  0.04  9183.25  1784  0.23  0.36  0.47   
50000  1786  9132.98  19.46  32.41  0.04  9182.20  1782  0.17  0.33  0.46   
55000  1782  9129.84  19.50  32.06  0.06  9183.13  1784  0.16  0.33  0.46   
60000  1782  9130.82  19.45  32.18  0.05  9183.13  1784  0.16  0.35  0.46   
65000  1780  9128.58  19.51  31.89  0.05  9182.46  1784  0.17  0.33  0.46   
70000  1781  9129.64  19.69  32.06  0.04  9182.46  1784  0.22  0.35  0.48   
75000  1779  9127.24  19.64  32.14  0.05  9180.81  1780  0.15  0.40  0.49   

      HIER:        
      CHILD OTHER  
5000   0.78  0.50  
10000  0.67  0.42  
15000  0.62  0.37  
20000  0.58  0.35  
25000  0.61  0.34  
30000  0.59  0.34  
35000  0.52  0.30  
40000  0.53  0.30  
45000  0.54  0.30  
50000  0.54  0.30  
55000  0.51  0.28  
60000  0.53  0.30  
65000  0.49  0.27  
70000  0.53  0.28  
75000  0.48  0.28

0 R: 1.000 P: 0.014 texts corpora english list types systems domain analysis languages dictionary
   1 R: 0.280 P: 0.097 features feature training performance learning accuracy models classification test classifier
     11 R: 0.080 P: 0.080 translation english source alignment target phrase translations parallel training pairs
     12 R: 0.103 P: 0.103 models training vector learning vectors performance dataset network embeddings features
   2 R: 0.215 P: 0.057 morphological pos character segmentation errors languages tagging arabic tags tag
     21 R: 0.076 P: 0.076 verb verbs noun syntactic argument semantic subject arguments head predicate
     22 R: 0.082 P: 0.082 tree parsing grammar parser dependency node trees rules parse algorithm
   3 R: 0.167 P: 0.053 human annotation evaluation level agreement study annotators test systems differences
     31 R: 0.053 P: 0.053 speech user dialogue utterance utterances speaker recognition spoken human systems
     32 R: 0.062 P: 0.062 topic s

KeyboardInterrupt: 

# analysis

In [15]:
def get_topic_year(sample_batches):
    probs_topics = []
    years = []
    for i, sample_batch in sample_batches:
        probs_topics_batch = sess.run(model.prob_topic, feed_dict=model.get_feed_dict(sample_batch, mode='test'))
        years_batch = [instance.year for instance in sample_batch]
        probs_topics += [probs_topics_batch]
        years += years_batch
    probs_topics = np.concatenate(probs_topics)
    years = np.array(years)

    topic_years = years.dot(probs_topics) / np.sum(probs_topics, 0)
    topic_year = {model.topic_idxs[i]: year for i, year in enumerate(topic_years)}
    return topic_year

In [18]:
sample_batches = get_batches(instances_train, config.batch_size)
topic_year = get_topic_year(sample_batches)
print_topic_year(sess, model, topic_freq_tokens=topic_freq_tokens, topic_year=topic_year)

0 Avg Year: 2008 texts corpora english list types systems domain analysis languages dictionary
   1 Avg Year: 2010 features feature training performance learning accuracy models classification test classifier
     11 Avg Year: 2010 translation english source alignment target phrase translations parallel training pairs
     12 Avg Year: 2013 models training vector learning vectors performance dataset network embeddings features
   2 Avg Year: 2007 morphological pos character segmentation errors languages tagging arabic tags tag
     21 Avg Year: 2006 verb verbs noun syntactic argument semantic subject arguments head predicate
     22 Avg Year: 2005 tree parsing grammar parser dependency node trees rules parse algorithm
   3 Avg Year: 2011 human annotation evaluation level agreement study annotators test systems differences
     31 Avg Year: 2007 speech user dialogue utterance utterances speaker recognition spoken human systems
     32 Avg Year: 2012 topic sentiment question questions an